In [ ]:
#基本的引入
import torch
import torchvision#为了转换图片格式
import torch.nn as nn
import torch.nn.functional as F
from pytorchtools import EarlyStopping#为了实现earlystopping
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
#为了minibatch的功能
import torch.utils.data as Data

torch.manual_seed(1)     # reproducible

In [ ]:
#改编版本
def train(model, device, loader,valid_loader, optimizer, epoch,loss_func):
    ###################
    # train the model #
    ###################
    model.train()
    for step, (batch_x, batch_y) in enumerate(loader):
        # here to train your model
        X, y = batch_x.to(device),  batch_y.to(device)
        optimizer.zero_grad()
        output = model(X)
        loss = loss_func(output, y)
        loss.backward()
        optimizer.step()
        
    ######################    
    # validate the model #
    ######################
    model.eval() # prep model for evaluation
    correct = 0
    for step, (batch_x, batch_y) in enumerate(valid_loader):
        X, y = batch_x.to(device),  batch_y.to(device)
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(X)
        # calculate the correct label
        pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
        correct += pred.eq(y.view_as(pred)).sum().item()
    
    
    print("Train Epoch: {} \tLoss: {:.6f}，Valid_Accuracy: {}/{} ({:.0f}%)\n".format(
        epoch,
        loss.item(),
        correct, len(valid_loader.dataset),
        100. * correct / len(valid_loader.dataset)
    ))


In [ ]:
def test(model, device, test_loader,loss_func):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for X, y in test_loader:
            X, y = X.to(device), y.to(device)
            output = model(X)
            test_loss += loss_func(output, y).item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(y.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
lr = 0.01
momentum = 0.5
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)#这个是和设备GPU/CPU相关的系数。
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)
loss_func = nn.CrossEntropyLoss()  # the target label is NOT an one-hotted
train_loader=minibatch(X_train,y_train)
valid_loader=minibatch(X_eval,y_eval)
test_loader=minibatch(X_test,y_test)

In [ ]:
epochs = 50
for epoch in range(1, epochs + 1):
    train(model, device, train_loader,valid_loader, optimizer, epoch,loss_func)
    test(model, device, test_loader,loss_func)

In [ ]:
# save_model = True
# if (save_model):
#     torch.save(model.state_dict(),"cnn.pt")